<a href="https://colab.research.google.com/github/AbrahamKong/Youtube_Summerizer/blob/main/Download_Youtube_Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install youtube-transcript-api
# !pip install --upgrade google-api-python-client
# !pip install openai==0.28

# # For Audio to text
# # install openai's whisper
# !pip install git+https://github.com/openai/whisper.git

# # update the packages
# !sudo apt update && sudo apt install ffmpeg

# # For Downloading Youtube Audio
# !pip install yt_dlp
# !pip install ffmpeg
# !mkdir youtubeaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 21.4 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-q8369mcb
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-q8369mcb
  Resolved https://github.com/openai/whisper.git to commit 8bc8860694949db53c42ba47ddc23786c2e02a8b
  Installing build depend

In [2]:
import openai
from googleapiclient.discovery import build

# Download Youtube WAV
from __future__ import unicode_literals
import yt_dlp
import ffmpeg
import sys

In [3]:
YOUTUBE_API_KEY = ''
OPENAI_API_KEY = 'sk-'

Function to extract youtube video ID from the URL

In [4]:
import re

def extract_video_id_from_url(url):
    """
    Extracts the YouTube video ID from a given URL.
    """
    regex = r"(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:[^\/\n\s]+\/\S+\/|(?:v|e(?:mbed)?)\/|\S*?[?&]v=)|youtu\.be\/)([a-zA-Z0-9_-]{11})"
    match = re.search(regex, url)
    return match.group(1) if match else None

Function to fetch the author, title and description of the youtube video

In [5]:
from googleapiclient.discovery import build

def get_youtube_video_details(url, youtube_api_key):
    """
    Fetches details of a YouTube video given its URL.
    """
    video_id = extract_video_id_from_url(url)

    # Initialize YouTube API client
    youtube = build('youtube', 'v3', developerKey=youtube_api_key)
    request = youtube.videos().list(part="snippet", id=video_id)
    response = request.execute()

    if 'items' in response and len(response['items']) > 0:
        title = response['items'][0]['snippet']['title']
        author = response['items'][0]['snippet']['channelTitle']
        description = response['items'][0]['snippet']['description']
        return title, author, description
    else:
        return "Video not found", "", ""

In [6]:
from youtube_transcript_api import YouTubeTranscriptApi, NoTranscriptFound

def get_video_transcript(video_url):
    try:
        video_id = extract_video_id_from_url(video_url)
        transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)

        # Fetch the transcript in English or the first available transcript
        if transcript_list.find_transcript(['en']):
            transcript = transcript_list.find_transcript(['en']).fetch()
        else:
            transcript = transcript_list[0].fetch()

        # Combine the text of the transcript
        combined_transcript = ' '.join([t['text'] for t in transcript])
        return combined_transcript
    except NoTranscriptFound:
        return "No transcript found for this video."

Function to summarize the youtube video using OpenAI API

In [7]:
import openai

def summarize_text(text, openai_api_key):
    """
    Summarizes the given text using OpenAI's GPT-3.
    """
    openai.api_key = openai_api_key

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Summarize the following text:\n\n{text}"}
        ],
        max_tokens=1500
    )

    return response['choices'][0]['message']['content'].strip()

def split_text_into_chunks(text, chunk_size=4000):
    """
    Splits text into smaller chunks, each with a maximum of chunk_size characters.
    """
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

def summarize_large_text(text, openai_api_key):
    """
    Splits a large text into chunks and summarizes each chunk.
    """
    chunks = split_text_into_chunks(text)
    summaries = [summarize_text(chunk, openai_api_key) for chunk in chunks]
    return ' '.join(summaries)



Function to download youtube audio

In [8]:
def download_youtube_audio(title, url):
  # Download options
  ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
    }],
    "outtmpl": f'youtubeaudio/{title}',  # Output template
  }

  # Function to download from URL
  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
      ydl.download([url])

Use the audio to speech-to-text

In [9]:
def speech_to_text(AUDIO_PATH):
  # Run Whisper AI:
  !whisper {AUDIO_PATH} --model medium

  # Additional arguments:
  !whisper -h

In [10]:
def generate_transcript(url):
  # Grab Youtube title to name the audio file
  title, author, description = get_youtube_video_details(url, YOUTUBE_API_KEY)
  AUDIO_NAME = title

  download_youtube_audio(title, url)

  audio_path = 'youtubeaudio/' + title

  speech_to_text(audio_path)



In [11]:
# Example

generate_transcript('https://youtu.be/_AnovXOJOaw')

[youtube] Extracting URL: https://youtu.be/_AnovXOJOaw
[youtube] _AnovXOJOaw: Downloading webpage
[youtube] _AnovXOJOaw: Downloading ios player API JSON
[youtube] _AnovXOJOaw: Downloading android player API JSON
[youtube] _AnovXOJOaw: Downloading player d23221b6
[youtube] _AnovXOJOaw: Downloading m3u8 information
[info] _AnovXOJOaw: Downloading 1 format(s): 251
[download] Destination: youtubeaudio/New AI: 6,000,000,000 Steps In 24 Hours!
[download] 100% of    7.06MiB in 00:00:00 at 26.45MiB/s  
[ExtractAudio] Destination: youtubeaudio/New AI: 6,000,000,000 Steps In 24 Hours!.wav
Deleting original file youtubeaudio/New AI: 6,000,000,000 Steps In 24 Hours! (pass -k to keep)
100%|█████████████████████████████████████| 1.42G/1.42G [00:16<00:00, 90.2MiB/s]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/whisper/audio.py", line 58, in load_audio
    out = run(cmd, capture_output=True, check=True).stdout
  File "/usr/lib/python3.10/subprocess.py", line 526, 

In [12]:
# # Example

# filedownload_youtube_audio('https://youtu.be/qijcT0LnqdU')

# speech_to_text('/content/youtubeaudio/看《首爾之春》氣到要測心跳？韓年輕世代起「心率挑戰」運動！這個影響韓國近代史的重大事件是什麼？ 【TODAY 看世界】.wav')

In [13]:
# speech_to_text('/content/youtubeaudio/看《首爾之春》氣到要測心跳？韓年輕世代起「心率挑戰」運動！這個影響韓國近代史的重大事件是什麼？ 【TODAY 看世界】.wav')